# Create users and manage permissions

We create two schemas, insert dummy data, then create two users with different privileges.

In [1]:
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    port=5432,
    dbname="exampledatabase",
    user="tristan",
    password="hunter2"
)

cur = conn.cursor()
cur.execute("CREATE SCHEMA IF NOT EXISTS schema1")
cur.execute("CREATE TABLE IF NOT EXISTS schema1.test (id SERIAL PRIMARY KEY, name TEXT);")
cur.execute("INSERT INTO schema1.test (name) VALUES (%s)", ("Alice",))
conn.commit()

cur.execute("CREATE SCHEMA IF NOT EXISTS schema2")
cur.execute("CREATE TABLE IF NOT EXISTS schema2.test (id SERIAL PRIMARY KEY, name TEXT);")
cur.execute("INSERT INTO schema1.test (name) VALUES (%s)", ("Bob",))
conn.commit()

See what we just created:

In [2]:
cur.execute("""
SELECT table_schema, table_name
FROM information_schema.tables
WHERE table_type = 'BASE TABLE'
  AND table_schema NOT IN ('pg_catalog', 'information_schema')
ORDER BY table_schema, table_name
""")

tables = cur.fetchall()
print("\nTables:")
for schema, table in tables:
    print(f"- {schema}.{table}")


Tables:
- schema1.test
- schema2.test


### Create two users

In [3]:
cur.execute("CREATE USER readonly_user WITH PASSWORD 'readonly_pass'")
cur.execute("CREATE USER readwrite_user WITH PASSWORD 'readwrite_pass'")
conn.commit()

### Permissions for user 1 (read only, on schema1)

In [4]:
# Grant CONNECT and SELECT on the database/table
cur.execute("GRANT CONNECT ON DATABASE exampledatabase TO readonly_user")
cur.execute("GRANT USAGE ON SCHEMA schema1 TO readonly_user")
cur.execute("GRANT SELECT ON schema1.test TO readonly_user")
conn.commit()

Remark 1: `cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA schema1 TO readonly_user")` is also valid to grant permission to all existing tables in a schema.

Remark 2: `cur.execute("ALTER DEFAULT PRIVILEGES IN SCHEMA schema1 GRANT SELECT ON TABLES TO readonly_user")` would be used to grant permission on future tables. 

### Permissions for user 2 (read/write on schema2)

In [5]:
cur.execute("GRANT CONNECT ON DATABASE exampledatabase TO readwrite_user")
cur.execute("GRANT USAGE ON SCHEMA schema2 TO readwrite_user")
cur.execute("GRANT SELECT, INSERT, UPDATE, DELETE ON schema2.test TO readwrite_user")
conn.commit()

### See who can do what

Remark 1: `c.relkind` can also be `v` for a view and `m` for a materialized view, a slightly longer query can list them too. 

Remark 2: functions, procedures, aggregates, and window functions are listed in `pg_proc`, so if you want to list them too along with their permissions etc you need a separate query into the catalogue.

In [6]:
cur.execute("""
SELECT
    n.nspname AS schema,
    c.relname AS table,
    r.rolname AS owner,
    g.grantee,
    g.privilege_type
FROM
    pg_class c
JOIN
    pg_namespace n ON n.oid = c.relnamespace
JOIN
    pg_roles r ON r.oid = c.relowner
JOIN
    information_schema.role_table_grants g
    ON g.table_name = c.relname AND g.table_schema = n.nspname
WHERE
    c.relkind = 'r'  -- only base tables
    AND n.nspname NOT IN ('pg_catalog', 'information_schema')
ORDER BY
    n.nspname, c.relname, g.grantee;
""")

rows = cur.fetchall()
print(f"{'Schema':<15} {'Table':<25} {'Owner':<20} {'Grantee':<20} {'Privilege':<15}")
print("-" * 95)
for schema, table, owner, grantee, privilege in rows:
    print(f"{schema:<15} {table:<25} {owner:<20} {grantee:<20} {privilege:<15}")

Schema          Table                     Owner                Grantee              Privilege      
-----------------------------------------------------------------------------------------------
schema1         test                      tristan              readonly_user        SELECT         
schema1         test                      tristan              tristan              SELECT         
schema1         test                      tristan              tristan              UPDATE         
schema1         test                      tristan              tristan              DELETE         
schema1         test                      tristan              tristan              INSERT         
schema1         test                      tristan              tristan              REFERENCES     
schema1         test                      tristan              tristan              TRIGGER        
schema1         test                      tristan              tristan              TRUNCATE       
sche

## Reconnect as an other user and list all tables again

In [7]:
conn.close()

In [8]:
conn = psycopg2.connect(
    host="localhost",
    port=5432,
    dbname="exampledatabase",
    user="readonly_user",
    password="readonly_pass"
)

cur = conn.cursor()

In [9]:
cur.execute("""
SELECT
    n.nspname AS schema,
    c.relname AS table,
    r.rolname AS owner,
    g.grantee,
    g.privilege_type
FROM
    pg_class c
JOIN
    pg_namespace n ON n.oid = c.relnamespace
JOIN
    pg_roles r ON r.oid = c.relowner
JOIN
    information_schema.role_table_grants g
    ON g.table_name = c.relname AND g.table_schema = n.nspname
WHERE
    c.relkind = 'r'  -- only base tables
    AND n.nspname NOT IN ('pg_catalog', 'information_schema')
ORDER BY
    n.nspname, c.relname, g.grantee;
""")

rows = cur.fetchall()
print(f"{'Schema':<15} {'Table':<25} {'Owner':<20} {'Grantee':<20} {'Privilege':<15}")
print("-" * 95)
for schema, table, owner, grantee, privilege in rows:
    print(f"{schema:<15} {table:<25} {owner:<20} {grantee:<20} {privilege:<15}")

Schema          Table                     Owner                Grantee              Privilege      
-----------------------------------------------------------------------------------------------
schema1         test                      tristan              readonly_user        SELECT         


This time, only `schema1` is visible!

In [10]:
conn.close()